In [1]:
!pip install -U transformers
!pip install -q pillow accelerate einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 16.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.5 MB/s eta 0:00:00


In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from PIL import Image

In [5]:
torch.set_default_device("cuda")

In [15]:
model = AutoModelForCausalLM.from_pretrained(
    "MILVLG/imp-v1-3b",
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(
    "MILVLG/imp-v1-3b",
    trust_remote_code=True
)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [21]:
text = """A chat between a curious user and an artificial intelligence assistant.
The assistant generates helpful and detailed answer after analyzing an image.
USER: <image>\nAnalyze the image. ASSISTANT:"""
image = Image.open("sample.jpg")

In [22]:
input_ids = tokenizer(text, return_tensors="pt").input_ids
image_tensor = model.image_preprocess(image)
output_ids = model.generate(
    input_ids,
    max_new_tokens=300,
    images=image_tensor,
    use_cache=True)[0]
print(tokenizer.decode(output_ids[input_ids.shape[1]:], skip_special_tokens=True).strip())

The image features a close-up view of a person's arm, showcasing a red rash or skin irritation. The rash appears to be quite large and covers a significant portion of the arm. The person's arm is also covered in small red spots, which may be related to the rash.

The person's arm is positioned in the center of the image, with the rash taking up most of the space. The redness and irritation are clearly visible, making it evident that the person is experiencing discomfort or skin issues.


In [9]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [10]:
!pip install fastapi==0.104.1 typing_extensions==4.8.0 gradio==3.41.0

In [11]:
import gradio as gr

In [17]:
def generate_answer(image):
  input_ids = tokenizer(text, return_tensors="pt").input_ids
  image_tensor = model.image_preprocess(image)
  output_ids = model.generate(
      input_ids,
      max_new_tokens=300,
      images=image_tensor,
      use_cache=True)[0]
  result = tokenizer.decode(output_ids[input_ids.shape[1]:], skip_special_tokens=True).strip()
  return result

In [18]:
# Create a Gradio interface
iface = gr.Interface(fn=generate_answer,
                     inputs=gr.inputs.Image(type="pil"),
                     outputs="text",
                     title="Multimodal Demo app using a Small Vision Model",
                     description="Upload an image to analyze.")

# Run the app
iface.launch(share=True, debug=True)

<ipython-input-18-1ea8a0ef6743>:3: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs=gr.inputs.Image(type="pil"),
<ipython-input-18-1ea8a0ef6743>:3: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Image(type="pil"),


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://b59a45ea012b465722.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1413: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/routes.py", line 488, in run_predict
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1435, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1107, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thre

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://b59a45ea012b465722.gradio.live
